Compute distances for distance KDE

In [1]:
# setup
import gridtools as gt
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
%matplotlib qt

from plotstyle import PlotStyle

s = PlotStyle()
dataroot = "../output/"
eventstats = pd.read_csv(dataroot + "eventstats.csv")
recs = gt.ListRecordings(dataroot)

# compute distance kde's
all_distances = []
dyad_distances = []
event_distances = []
n_all = 0
n_interact = 0
n_events = 0

for recording in recs.recordings:
    datapath = recs.dataroot + recording + "/"
    grid = gt.GridTracks(datapath, finespec=False)
    events = pd.read_csv(datapath + "events.csv")

    # get all distances
    n_all += len(grid.ids)
    dyad_ids = gt.utils.unique_combinations1d(grid.ids)
    for ids in dyad_ids:
        dyad = gt.Dyad(grid, ids)
        if dyad.overlap is True:
            all_distances.extend(dyad.dpos.tolist())

    # get unique interacting dyad distances
    dyads = []
    uniq1 = list(np.unique(events.id1))
    uniq2 = list(np.unique(events.id2))
    all_ids = np.unique(np.append(uniq1, uniq2))
    n_interact += len(all_ids)

    for idx in events.index:
        dy = sorted([events.id1[idx], events.id2[idx]])
        dyads.append(dy)
    unique_dyads = [list(x) for x in set(tuple(x) for x in dyads)]
    for ids in unique_dyads:
        dyad = gt.Dyad(grid, ids)
        if dyad.overlap is True:
            dyad_distances.extend(dyad.dpos.tolist())

    # get distances during events
    for idx in events.index:
        dyad = gt.Dyad(grid, [int(events.id1[idx]), int(events.id2[idx])])
        start = gt.utils.find_closest(dyad.times, float(events.start[idx]))
        stop = gt.utils.find_closest(dyad.times, float(events.stop[idx]))
        event_distances.extend(dyad.dpos[start:stop].tolist())
        n_events += 1

# convert to numpy arrays
all_distances = np.asarray(all_distances)
dyad_distances = np.asarray(dyad_distances)
event_distances = np.asarray(event_distances)

[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-20-18_49/
[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-18-19_22/
[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-16-18_45/
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ Dyad.__init__ ] Supplied identities do not share a common overlap!
[ GridTracks.__init__ ] No temperature an light data found in directory ../output/2016-04-09-22_25/.
[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-09-22

In [2]:
# make kdes for all
xlims_dpos = [0, 400]
bandwidth = 10

# compute kde
kde_d_all = gt.utils.kde1d(all_distances, bandwidth, xlims_dpos)
kde_d_interact = gt.utils.kde1d(dyad_distances, bandwidth, xlims_dpos)
kde_d_events = gt.utils.kde1d(event_distances, bandwidth, xlims_dpos)

kde_d_all = dict(x=kde_d_all[0], y = kde_d_all[1])
kde_d_interact = dict(x=kde_d_interact[0], y = kde_d_interact[1])
kde_d_events = dict(x=kde_d_events[0], y = kde_d_events[1])

kde_d_all = pd.DataFrame(kde_d_all)
kde_d_interact = pd.DataFrame(kde_d_interact)
kde_d_events = pd.DataFrame(kde_d_events)

kde_d_all.to_csv("../processed_data/kde_d_all.csv")
kde_d_interact.to_csv("../processed_data/kde_d_interact.csv")
kde_d_events.to_csv("../processed_data/kde_d_events.csv")

computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9979630526210534
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9934363360970784
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9558349107146368


Export up-modulation during Events for initiators as reactors (data and KDE)

In [3]:
resting_init = []
resting_react = []
max_init = []
max_react = []
diff_resting_withstatus = []
diff_resting = []

for idx in eventstats.index:
    init_id = float(eventstats.initiator[idx])

    diff_resting.append(abs(eventstats["restingf_id1"][idx] - eventstats["restingf_id2"][idx]))
    if np.isnan(init_id) == False:
        
        if int(init_id) == eventstats.id1[idx]:
            resting_init.append(float(eventstats.restingf_id1[idx]))
            max_init.append(float(eventstats.maxeventf_id1[idx]))
            resting_react.append(float(eventstats.restingf_id2[idx]))
            max_react.append(float(eventstats.maxeventf_id2[idx]))
        
        elif int(init_id) == eventstats.id2[idx]:
            resting_init.append(float(eventstats.restingf_id2[idx]))
            max_init.append(float(eventstats.maxeventf_id2[idx]))
            resting_react.append(float(eventstats.restingf_id1[idx]))
            max_react.append(float(eventstats.maxeventf_id1[idx]))
        
        diff_resting_withstatus.append(abs(eventstats["restingf_id1"][idx] - eventstats["restingf_id2"][idx]))

resting_init = np.asarray(resting_init)
resting_react = np.asarray(resting_react)
max_init = np.asarray(max_init)
max_react = np.asarray(max_react)

resting_initreact_diff = resting_init - resting_react
kde_f_restinginitreact_diff = gt.utils.kde1d(resting_initreact_diff, 2)

init_amp = max_init - resting_init
react_amp = max_react - resting_react

xlims = [0, 50]
kde_react_amp = gt.utils.kde1d(react_amp, 1, xlims)
kde_init_amp = gt.utils.kde1d(init_amp, 1, xlims)

event_f_increase = dict(init=init_amp, react=react_amp)
event_f_increase = pd.DataFrame(event_f_increase)
event_f_increase.to_csv("../processed_data/event_f_increase.csv")

kde_react_increase = dict(x=kde_react_amp[0], y=kde_react_amp[1])
kde_react_increase = pd.DataFrame(kde_react_increase)
kde_react_increase.to_csv("../processed_data/kde_react_increase.csv")

kde_init_increase = dict(x=kde_init_amp[0], y = kde_init_amp[1])
kde_init_increase = pd.DataFrame(kde_init_increase)
kde_init_increase.to_csv("../processed_data/kde_init_increase.csv")

computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9328044712312922
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9999999861049931
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9999999977347609


Get delta f of dyads before and during events

In [4]:
diff_resting = abs(eventstats["restingf_id1"] - eventstats["restingf_id2"])

diff_f_before_vs_during_event = dict(before=diff_resting, during=eventstats["diff_medianevent"])
diff_f_before_vs_during_event = pd.DataFrame(diff_f_before_vs_during_event)
diff_f_before_vs_during_event.to_csv("../processed_data/diff_f_before_vs_during_event.csv")

Extract resting initiator ID - resting reactor ID

In [5]:
resting_init = []
resting_react = []
max_init = []
max_react = []
diff_resting_withstatus = []
diff_resting = []

for idx in eventstats.index:
    init_id = float(eventstats.initiator[idx])

    diff_resting.append(abs(eventstats["restingf_id1"][idx] - eventstats["restingf_id2"][idx]))
    if np.isnan(init_id) == False:
        
        if int(init_id) == eventstats.id1[idx]:
            resting_init.append(float(eventstats.restingf_id1[idx]))
            max_init.append(float(eventstats.maxeventf_id1[idx]))
            resting_react.append(float(eventstats.restingf_id2[idx]))
            max_react.append(float(eventstats.maxeventf_id2[idx]))
        
        elif int(init_id) == eventstats.id2[idx]:
            resting_init.append(float(eventstats.restingf_id2[idx]))
            max_init.append(float(eventstats.maxeventf_id2[idx]))
            resting_react.append(float(eventstats.restingf_id1[idx]))
            max_react.append(float(eventstats.maxeventf_id1[idx]))
        
        diff_resting_withstatus.append(abs(eventstats["restingf_id1"][idx] - eventstats["restingf_id2"][idx]))

resting_init = np.asarray(resting_init)
resting_react = np.asarray(resting_react)
max_init = np.asarray(max_init)
max_react = np.asarray(max_react)

resting_initreact_diff = resting_init - resting_react
kde_f_restinginitreact_diff = gt.utils.kde1d(resting_initreact_diff, 2)

resting_initreact_diff = dict(diff_resting = resting_initreact_diff)
resting_initreact_diff = pd.DataFrame(resting_initreact_diff)
resting_initreact_diff.to_csv("../processed_data/diff_f_init_react.csv")

kde_f_restinginitreact_diff = dict(x=kde_f_restinginitreact_diff[0], y=kde_f_restinginitreact_diff[1])
kde_f_restinginitreact_diff = pd.DataFrame(kde_f_restinginitreact_diff)
kde_f_restinginitreact_diff.to_csv("../processed_data/kde_f_restinginitreact_diff.csv")


computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9328044712312922


Extract physical interaction events for initiators, reactors and total and appropriate time axis.

In [6]:
all_veloc = []
dyad_veloc = []
event_veloc = []
event_interactions_init = []
event_interactions_react = []
event_interactions = []
event_interactions_ts = []
event_durations = []

for recording in recs.recordings:
    datapath = recs.dataroot + recording + "/"
    grid = gt.GridTracks(datapath, finespec=False)
    events = pd.read_csv(datapath + "events.csv")

    # get all distances
    for track_id in grid.ids:
        t = grid.times[grid.idx_v[grid.ident_v == track_id]]
        x = grid.xpos_smth[grid.ident_v == track_id]
        y = grid.ypos_smth[grid.ident_v == track_id]
        v = gt.utils.velocity2d(t, x, y)
        all_veloc.extend(v.tolist())

    # get unique interacting dyad distances
    dyads = []
    uniq1 = list(np.unique(events.id1))
    uniq2 = list(np.unique(events.id2))
    all_ids = np.unique(np.append(uniq1, uniq2))
    for track_id in all_ids:
        t = grid.times[grid.idx_v[grid.ident_v == track_id]]
        x = grid.xpos_smth[grid.ident_v == track_id]
        y = grid.ypos_smth[grid.ident_v == track_id]
        v = gt.utils.velocity2d(t, x, y)
        dyad_veloc.extend(v.tolist())

    # get data during events
    for idx in events.index:

        # extract all positions for kde
        dyad = gt.Dyad(grid, [int(events.id1[idx]), int(events.id2[idx])])
        start = gt.utils.find_closest(dyad.times, float(events.start[idx]))
        stop = gt.utils.find_closest(dyad.times, float(events.stop[idx]))

        t = dyad.times[start:stop]
        x1 = dyad.xpos_smth_id1[start:stop]
        x2 = dyad.xpos_smth_id2[start:stop]
        y1 = dyad.xpos_smth_id1[start:stop]
        y2 = dyad.xpos_smth_id2[start:stop]

        v1 = gt.utils.velocity2d(t, x1, y1)
        v2 = gt.utils.velocity2d(t, x2, y2)

        v = np.append(v1, v2)
        event_veloc.extend(v.tolist())

        # get event edurations
        dur = dyad.times[stop]-dyad.times[start]

        # extract social events for event triggered interactions

        # get approx 5 minutes before event start and 10 minutes after event start
        start2 = start - 1000
        stop2 = start + 2000

        t_rasterplot = dyad.times[start2:stop2] - dyad.times[start]
        
        peakprom = 1
        maxd = 25

        peaks1, peaks2 = gt.utils.find_interactions(dyad, start2, stop2, maxd=maxd, peakprom=peakprom)
        peaks = np.append(peaks1, peaks2)

        # detect interactions for intiators and reactors
        if events.init[idx] == events.id1[idx]:
            peaks_init, peaks_react = gt.utils.find_interactions(dyad, start2, stop2, maxd=maxd, peakprom=peakprom)
            event_durations.append(dur)
        elif events.init[idx] == events.id2[idx]:
            peaks_react, peaks_init = gt.utils.find_interactions(dyad, start2, stop2, maxd=maxd, peakprom=peakprom)
            event_durations.append(dur)
        else:
            continue

        # append to list    
        event_interactions_init.append(peaks_init)
        event_interactions_react.append(peaks_react)
        event_interactions.append(peaks)


# convert to numpy arrays
t_rasterplot = np.asarray(t_rasterplot)

# sort events by syncrhonized duration in reverse order, then sort duration
event_interactions_init = [x for _, x in sorted(zip(event_durations, event_interactions_init), key=lambda pair: pair[0], reverse=True)]
event_interactions_react = [x for _, x in sorted(zip(event_durations, event_interactions_react), key=lambda pair: pair[0], reverse=True)]
event_interactions = [x for _, x in sorted(zip(event_durations, event_interactions), key=lambda pair: pair[0], reverse=True)]

# sort event durations as well
event_durations.sort(reverse=True)

[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-20-18_49/
[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-18-19_22/
[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-16-18_45/
[ GridTracks.__init__ ] No temperature an light data found in directory ../output/2016-04-09-22_25/.
[ GridTracks.__init__ ] No grid metadata found in directory ../output/2016-04-09-22_25/
[ Dyad.__init__  ] GridTracks instance has no temperature and light arrays.
[ Dyad.__init__  ] GridTracks instance has no temperature and light arrays.
[ Dyad.__init__  ] GridTracks instance has no temperature and light arrays.


Make kde for interaction events rasterplot, i.e. event onset triggered interactions and transform peak indices to timestamps

In [7]:
# make kde for interaction events rasterplot, i.e. event onset triggered interactions
bandwidth = 10
#event_interactions_flat = np.concatenate(event_interactions)

# flatten data
init_flattened = np.concatenate(event_interactions_init)
react_flattened = np.concatenate(event_interactions_react)
all_flattened = np.concatenate(event_interactions)

# make flattened timestamps
init_ts = t_rasterplot[init_flattened]
react_ts = t_rasterplot[react_flattened]
all_ts = t_rasterplot[all_flattened]

# calculate separate kdes:
init_raster_kde = gt.utils.kde1d(init_ts, bandwidth, [np.min(t_rasterplot), np.max(t_rasterplot)])
react_raster_kde = gt.utils.kde1d(react_ts, bandwidth, [np.min(t_rasterplot), np.max(t_rasterplot)])
all_raster_kde = gt.utils.kde1d(react_ts, bandwidth, [np.min(t_rasterplot), np.max(t_rasterplot)])

# save kdes to file
init_raster_kde = dict(x=init_raster_kde[0], y=init_raster_kde[1])
react_raster_kde = dict(x=react_raster_kde[0], y=react_raster_kde[1])
all_raster_kde = dict(x=all_raster_kde[0], y=all_raster_kde[1])

init_raster_kde = pd.DataFrame(init_raster_kde)
react_raster_kde = pd.DataFrame(react_raster_kde)
all_raster_kde = pd.DataFrame(all_raster_kde)

init_raster_kde.to_csv('../processed_data/kde_raster_init.csv')
react_raster_kde.to_csv('../processed_data/kde_raster_react.csv')
all_raster_kde.to_csv('../processed_data/kde_raster_all.csv')

# make timestamp matrices
raster_ts_init = []
raster_ts_react = []
raster_ts_all = []
for init, react, interact in zip(event_interactions_init, event_interactions_react, event_interactions):
    rti = t_rasterplot[init]
    rtr = t_rasterplot[react]
    rta = t_rasterplot[interact]
    raster_ts_init.append(rti)
    raster_ts_react.append(rtr)
    raster_ts_all.append(rta)

# save timestamp matrices
np.save("../processed_data/raster_ts_init.npy", raster_ts_init, allow_pickle=True)
np.save("../processed_data/raster_ts_react.npy", raster_ts_react, allow_pickle=True)
np.save("../processed_data/raster_ts_all.npy", raster_ts_all, allow_pickle=True)



computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9899438573306721
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9941783933566748
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9941783933566748


/home/weygoldt/Data/uni/efish/env/lib/python3.10/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Jigsaw and resampling

In [8]:
# extract event df
bandwidth = 10

nsamples = len(all_ts) # all timestamps, i.e. all interaction events
nresamples = 10000

all_ts = sorted(all_ts)
# plt.eventplot(all_ts)

diffs = np.diff(all_ts, prepend = all_ts[0])

print(f'timestamp length: {len(all_ts)}')
print(f'diffs length: {len(diffs)}')

print(f'timestamps: {all_ts[0:10]}')
print(f'diffs: {diffs[0:10]}')

kdes = [] # collect kdes here

# resample
for i in range(nresamples):

    # shuffle randomly
    np.random.shuffle(diffs)

    # make shuffled to timestamps starting with first time point
    randevents = np.concatenate(([t_rasterplot[0]], diffs)).cumsum()

    # compute kde
    randkde = gt.utils.kde1d(randevents, bandwidth)

    # stash kde
    kdes.append(randkde[1])

# compute median
kdes = np.asarray(kdes)
plt.plot(all_raster_kde['x'], all_raster_kde['y'], color = 'red', lw = 3)

baseline_medians = np.median(kdes, axis = 0)
baseline_q5s = np.quantile(kdes, 0.05, axis=0)
baseline_q95s = np.quantile(kdes, 0.95, axis=0)



timestamp length: 561
diffs length: 561
timestamps: [-326.6954499999956, -325.3846499999963, -319.15889999999854, -316.537849999997, -316.537849999997, -315.5547499999957, -315.2270499999977, -314.89934999999605, -310.9669499999982, -310.9669499999982]
diffs: [0.      1.3108  6.22575 2.62105 0.      0.9831  0.3277  0.3277  3.9324
 0.     ]
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9890245154568117
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9788257088062856
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9907736983653211
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9927313035935447
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9906236640603887
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9953080671696888
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9911244508194998
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.98653611435

Jackknife

In [9]:
bandwidth = 10

nsamples = len(all_ts) # all timestamps, i.e. all interaction events
nresamples = 10000

all_ts = sorted(all_ts)
# plt.eventplot(all_ts)

kdes = []
subsetsize = int(0.9 * len(all_ts))

# resample
for i in range(nresamples):

    # compute kde
    randkde = gt.utils.kde1d(np.random.choice(all_ts, subsetsize, replace = False), bandwidth)

    # stash kde
    kdes.append(randkde[1])

estimated_medians = np.median(kdes, axis = 0)
estimated_q5s = np.quantile(kdes, 0.05, axis=0)
estimated_q95s = np.quantile(kdes, 0.95, axis=0)

computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9897873051105137
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9904640415246939
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9896954420051275
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9915888110204854
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.989400615350524
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9907008452705262
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9893832219488715
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9894200100407907
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9896412323553536
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9901214120107876
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9905177563830252
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9904566753988837
compu

In [10]:
plt.plot(all_raster_kde['x'], estimated_medians)
plt.fill_between(all_raster_kde['x'], estimated_q5s, estimated_q95s, alpha =0.4)

plt.plot(all_raster_kde['x'], baseline_medians, color = 'black', lw = 1.5)
plt.fill_between(all_raster_kde['x'], baseline_q5s, baseline_q95s, color = 'lightgray', zorder = -100)

In [11]:
# export the median, quartiles, etc to file

bootstrap_kde = dict(
    x = all_raster_kde['x'], 
    baseline_median = baseline_medians,
    baseline_q5 = baseline_q5s, 
    baseline_q95 = baseline_q95s,
    estimated_median = estimated_medians, 
    estimated_q5 = estimated_q5s,
    estimated_q95 = estimated_q95s
)

bootstrap_kde = pd.DataFrame(bootstrap_kde)
bootstrap_kde.to_csv('../processed_data/kde_bootstrap.csv')

Export distance KDE

In [12]:
# make kdes for all
bandwidth = 0.01

# convert to numpy arrays
all_veloc = np.asarray(all_veloc)
dyad_veloc = np.asarray(dyad_veloc)
event_veloc = np.asarray(event_veloc)

# compute kde of velocities
kde_v_all = gt.utils.kde1d(all_veloc[~np.isnan(all_veloc)], bandwidth)
kde_v_interact = gt.utils.kde1d(dyad_veloc[~np.isnan(dyad_veloc)], bandwidth)
kde_v_events = gt.utils.kde1d(event_veloc[~np.isnan(event_veloc)], bandwidth)

kde_v_all = dict(x=kde_v_all[0], y = kde_v_all[1])
kde_v_interact = dict(x=kde_v_interact[0], y = kde_v_interact[1])
kde_v_events = dict(x=kde_v_events[0], y = kde_v_events[1])

kde_v_all = pd.DataFrame(kde_v_all)
kde_v_interact = pd.DataFrame(kde_v_interact)
kde_v_events = pd.DataFrame(kde_v_events)

kde_v_all.to_csv("../processed_data/kde_v_all.csv")
kde_v_interact.to_csv("../processed_data/kde_v_interact.csv")
kde_v_events.to_csv("../processed_data/kde_v_events.csv")

computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 1.0118666359165367
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9968494797949562
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9911281499297891
